# Select and Train a Model

이제 모델을 학습시킬 준비가 되었다.

먼저, 앞 절에서 수행한 내용들을 통해 데이터를 준비하자.

In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit

dataset_root = os.path.join(os.getcwd(), "datasets")
housing_path = os.path.join(dataset_root, "housing")

def load_housing_data(housing_path=housing_path):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)


housing = load_housing_data()
housing["income_cat"] = np.ceil(housing["median_income"] / 1.5)
housing["income_cat"].where(housing["income_cat"] < 5, 5.0, inplace=True)

split = StratifiedShuffleSplit(n_splits=1,
                               test_size=0.2,
                               random_state=42)

for train_index, test_index in split.split(housing, housing["income_cat"]):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]
    

for set_ in (strat_train_set, strat_test_set):
    set_.drop("income_cat", axis=1, inplace=True)

In [2]:
housing = strat_train_set.drop("median_house_value", axis=1)
housing_labels = strat_train_set["median_house_value"].copy()

housing_num = housing.drop("ocean_proximity", axis=1)
housing_cat = housing[["ocean_proximity"]]

In [3]:
from sklearn.base import BaseEstimator, TransformerMixin

rooms_idx, bedrooms_idx, population_idx, households_idx = 3, 4, 5, 6

class CombinedFeaturesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room=True):
        self.add_bedrooms_per_room = add_bedrooms_per_room
    
    def fit(self, X, y=None):
        # Nothing else to do
        return self
    
    def transform(self, X, y=None):
        rooms_per_household = X[:, rooms_idx]/X[:, households_idx]
        population_per_household = X[:, population_idx]/X[:, households_idx]
        # bedrooms_per_room은 선택
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_idx]/X[:, rooms_idx]
            return np.c_[X,
                         rooms_per_household,
                         population_per_household,
                         bedrooms_per_room]
        else:
            return np.c_[X,
                         rooms_per_household,
                         population_per_household]

In [4]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('features_adder', CombinedFeaturesAdder()),
    ('std_scaler', StandardScaler())
])

In [5]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

num_features = list(housing_num)
cat_features = ["ocean_proximity"]

full_pipeline = ColumnTransformer([
    ("num", num_pipeline, num_features),
    ("cat", OneHotEncoder(), cat_features),
])

In [6]:
housing_prepared = full_pipeline.fit_transform(housing)

## Training and Evaluating on the Training set

Linear Regression model로 학습을 시켜보자.

In [7]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(housing_prepared, housing_labels)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

이제 training set의 instance 중 5개에서 prediction을 해보자.

In [8]:
some_data = housing.iloc[:5]
some_labels = housing_labels.iloc[:5]
some_data_prepared = full_pipeline.transform(some_data)

print("Predictions:", lin_reg.predict(some_data_prepared))
print("Labels:", list(some_labels))

Predictions: [210644.60459286 317768.80697211 210956.43331178  59218.98886849
 189747.55849879]
Labels: [286600.0, 340600.0, 196900.0, 46300.0, 254500.0]


전체 training set에서 RMSE를 측정해보자.

In [9]:
from sklearn.metrics import mean_squared_error

housing_predictions = lin_reg.predict(housing_prepared)
lin_mse = mean_squared_error(housing_labels, housing_predictions)
lin_rmse = np.sqrt(lin_mse)

print("RMSE:", lin_rmse)

RMSE: 68628.19819848923


`median_housing_values`의 범위는 120000달러에서 265000달러이므로 68628달러의 error는 좋지 못하다.

위의 결과는 training data에 underfitting한 경우이다. 이는 feature가 prediction에 충분한 정보를 제공하지 못하거나 model이 충분히 강력하지 못한 경우에 발생한다.

앞의 절에서 배웠듯이, underfitting을 해결하기 위해서는 다음과 같은 방법이 있다.
- 더 강력한 모델 사용
- 더 좋은 feature 주입
- 모델의 제한(regularization) 줄이기

여기서 사용한 모델은 regularization을 사용하지 않았으므로, 이를 제외한 2가지 방법을 시도해보자.

첫번째 방법인 더 강력한 모델을 사용해보자.

`DecisionTreeRegressor`는 복잡한 non-linear 관계를 찾을 수 있는 강력한 모델이다.

In [10]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor()
tree_reg.fit(housing_prepared, housing_labels)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [11]:
housing_predictions = tree_reg.predict(housing_prepared)
tree_mse = mean_squared_error(housing_labels, housing_predictions)
tree_rmse = np.sqrt(tree_mse)
print("RMSE:", tree_mse)

RMSE: 0.0


결과가 0이 나왔다. 이는 모델이 overfitting되었다는 것을 의미한다.

모델을 출시할 정도의 준비가 되지 않았다면, test set을 사용하는 것은 바람직하지 않다. training set에서 validation set을 분리해서 모델을 평가해보자.

## Better Evaluation Using Cross-Validation

`train_test_split`을 사용해 training set을 작은 training set과 validation set으로 나누어 학습 및 평가하는 것은 어렵지 않으며 잘 동작한다.

하지만 더 좋은 방법은 **K-fold cross-validation**이다.

이 방법은 training set을 무작위로 10개의 다른 subset(*fold*)로 분리해서 총 10번의 training 및 evaluation을 하는데, 매 수행마다 다른 fold 1개를 선택해 evaluation에 사용하고, 나머지 9개 fold를 training에 사용한다. 따라서, 총 10개의 evaluation score가 결과로 나오게 된다.

사이킷런에서는 `cross_val_score`를 통해 다음과 같이 수행할 수 있다.

In [12]:
from sklearn.model_selection import cross_val_score
'''
사이킷런에서는 cost function(낮을수록 좋음)이 아닌
utility function(클수록 좋음)을 사용하므로 결과값이 음으로 나오게 됨
'''
scores = cross_val_score(tree_reg, housing_prepared, housing_labels,
                         scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-scores)

def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())
    

display_scores(tree_rmse_scores)

Scores: [69220.17690295 65806.27871417 71479.75172819 69114.3012216
 71884.84240573 74889.45102715 71870.58600403 70307.66012319
 74494.42833838 68448.34666312]
Mean: 70751.58231284973
Standard deviation: 2630.2359821332348


Cross validation을 사용하면, 모델의 성능과 더불어 얼마나 평가가 정확한지(mean, std)도 알 수 있다.

위 결과를 통해, DecisionTreeRegressor는 약 70751 $\pm$ 2630의 rmse score를 가진다다는 것을 알 수 있다. 

Linear Regression에도 이를 적용해보자.

In [13]:
lin_scores = cross_val_score(lin_reg, housing_prepared, housing_labels,
                             scoring="neg_mean_squared_error", cv=10)
lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)

Scores: [66782.73843989 66960.118071   70347.95244419 74739.57052552
 68031.13388938 71193.84183426 64969.63056405 68281.61137997
 71552.91566558 67665.10082067]
Mean: 69052.46136345083
Standard deviation: 2731.674001798346


Linear Regression은 약 69052 $\pm$ 2731가 나왔다.

따라서, Linear Regression model이 Decision Tree model보다 더욱 성능이 좋다는 것을 알 수 있다.

마지막으로, `RandomForestRegressor`를 사용해보자.

Random Forest는 무작위로 feature들의 subset을 여러개 만들고 각각에 Decision Tree를 학습시킨 후 이들의 prediction을 평균내는 방식이다.

이처럼 여러 모델을 결합하여 하나의 모델로 만드는 것을 **Ensemble Learning**이라고 한다. 이는 머신러닝 알고리즘의 성능을 극대화하는 좋은 방법이다.

In [14]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor()
forest_reg.fit(housing_prepared, housing_labels)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [15]:
forest_scores = cross_val_score(forest_reg, housing_prepared, housing_labels,
                             scoring="neg_mean_squared_error", cv=10)
forest_rmse_scores = np.sqrt(-forest_scores)
display_scores(forest_rmse_scores)

Scores: [49510.1225241  47348.69818895 49723.15397471 52510.10624746
 49359.41961918 53749.31363372 48881.04756667 48192.60427935
 52540.94074958 50008.69239619]
Mean: 50182.40991799097
Standard deviation: 1968.6749296429166


Random Forest는 약 50182 $\pm$ 1968으로 가장 위 두 모델보다 좋은 결과를 냈다.

하지만 여전히 training set보다 validation set에서 더욱 좋지 못한 성능을 낸다는 것은 overfitting 된다는 것이다. 따라서, 단순한 모델 / regularization / training data 추가 등을 고려할 수 있지만, 지금은 Random Forest를 깊이 파고들기 보다는 다른 머신러닝 모델들(여러 kernel의 SVM, Neural Network 등)을 hyperparameter tuning없이 시도해보며 몇개(2~5개)의 괜찮은 모델을 선정하는 것이 우선이다.

그 이후에 선정된 모델들에 대해 hyperparameter tuning을 시도해야한다.

사이킷런의 모델은 파이썬의 `pickle` 모듈을 사용하거나 `sklearn.externals.joblib`을 통해 저장할 수 있다. 

다음은 numpy array의 serializing에 더욱 효율적인 `sklearn.externals.joblib`으로 Random Forest 모델을 저장하고 불러오는 예이다.

In [17]:
from sklearn.externals import joblib

joblib.dump(forest_reg, "./saved_models/my_model.pkl")

/home/young/anaconda3/envs/tf_2/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


['./saved_models/my_model.pkl']

In [19]:
loaded_forest_reg = joblib.load("./saved_models/my_model.pkl")
loaded_forest_scores = cross_val_score(loaded_forest_reg, housing_prepared, housing_labels,
                             scoring="neg_mean_squared_error", cv=10)
loaded_forest_rmse_scores = np.sqrt(-loaded_forest_scores)
display_scores(loaded_forest_rmse_scores)

Scores: [49396.09319498 47253.06877769 49997.39841391 52351.87339577
 49626.00931366 53682.01212858 48931.25989942 47907.65294212
 53272.67641473 50267.81816159]
Mean: 50268.58626424615
Standard deviation: 2066.872164376048
